[![Open Rendered Output](https://img.shields.io/badge/Rendered%20Output-Open-blue?logo=link&logoColor=white)](https://htmlpreview.github.io/?https://raw.githubusercontent.com/MeteoSwiss/nwp-fdb-polytope-demo/main/notebooks/snapshots/feature_time_series.html)


# Time Series Feature: Time Series at Zurich Airport

This notebook demonstrates efficient geolocation and time-series access using Polytope feature extraction. By retrieving only the requested grid points, the I/O is greatly reduced and workflows can be considerably sped up.

<div style="text-align:center;">
  <img src="./images/t_2m_time_series.png" style="width:50%;"/>
</div>

## Installation
Follow the instructions in [README.md](https://github.com/MeteoSwiss/nwp-fdb-polytope-demo/blob/main/README.md#Installation-1) to install the necessary dependencies.

## Configuring Access to Polytope
To access ICON data via MeteoSwiss's Polytope, you need a Polytope offline token provided by MeteoSwiss. If you do not already have a token, you can request one [here](https://meteoswiss.atlassian.net/wiki/spaces/IW2/pages/327780397/Polytope#Offline-token-authentication). Then, create a new `config.yml` file based on [`config_example.yml`](config_example.yml), and replace <meteoswiss_key> with your access token there.

In [ ]:
import os
import yaml

def load_config(path="config.yml"):
    if not os.path.exists(path):
        raise FileNotFoundError("Missing config.yml. Please create one based on config_example.yml.")
    with open(path, "r") as f:
        return yaml.safe_load(f)

config = load_config()

# ICON-CSCS Polytope credentials
os.environ["POLYTOPE_USER_KEY"] = config["meteoswiss"]["key"]
os.environ["POLYTOPE_ADDRESS"] = "https://polytope-depl.mchml.cscs.ch"

# To clear the environment variable
os.environ["POLYTOPE_USER_EMAIL"] = ""

## Select the geolocation
We’ll request a time series at **Zurich Airport** (WGS84 coordinates).

In [ ]:
zrh_geo_point = (8.565074, 47.453928)  # (longitude, latitude) in WGS84

## Rotate the point

Given that the data source accessed by Polytope is stored on a rotated grid, it is necessary to provide Polytope with the point in rotated coordinates, using a South Pole rotation with a reference of longitude 10° and latitude of -43°.

> **NOTE:** `transform_point()` expects `(longitude, latitude)`.

In [ ]:
import cartopy.crs as ccrs

geo_crs = ccrs.PlateCarree()
rotated_crs = ccrs.RotatedPole(pole_longitude=190, pole_latitude=43)

# Convert the point from geographic to rotated coordinates
geo_lon, geo_lat = zrh_geo_point
rot_lon, rot_lat = rotated_crs.transform_point(geo_lon, geo_lat, geo_crs)
rotated_point = (rot_lon, rot_lat)

## Select the run date and time
The FDB containing real-time data typically **holds only the most recent day of forecasts**. The next cell computes a valid run time automatically.

In [ ]:
from datetime import datetime, timedelta

now = datetime.now()
past_time = now - timedelta(hours=12)                # use a recent past cycle
rounded_hour = (past_time.hour // 6) * 6             # round down to 6-hour cycles
rounded_time = past_time.replace(hour=rounded_hour, minute=0, second=0, microsecond=0)

date = rounded_time.strftime('%Y%m%d')
time = rounded_time.strftime('%H%M')
date, time

## Define the feature
Request a **time series at one point** (`rotated_point`) over **steps 0–120**. ICON-CH2-EPS is hourly in this range (use 0–33 for ICON-CH1-EPS).

> **NOTE:** If points are `(longitude, latitude)`, set `axes=["longitude", "latitude"]`.

In [ ]:
feature = {
    "type": "timeseries",
    "points": [rotated_point],      # single point in the rotated grid (lon, lat)
    "time_axis": "step",
    "range": {"start": 0, "end": 120},   # hourly steps 0..120
    "axes": ["longitude", "latitude"]    # order must match the point tuple
}

## Define the request
Use [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/)'s `mars.Request` for a clean, validated API. This example fetches **2-m temperature** from **ICON-CH2-EPS** at the **surface**, for **ensemble member 1** (`number=1`), at the selected run date/time.
- `type="pf"` — perturbed member (requires `number`).
- `type="cf"` — control forecast (no `number`).

In [ ]:
from meteodatalab import mars

request = mars.Request(
    param="T_2M",
    date=date,
    time=time,
    model=mars.Model.ICON_CH2_EPS,
    levtype=mars.LevType.SURFACE,
    type="pf",
    number=1,
    feature=feature
)

## Data retrieval
Load the data with [earthkit.data](https://earthkit-data.readthedocs.io/en/latest/) and convert it to an [xarray.Dataset](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html).

In [ ]:
import earthkit.data as ekd
ds = ekd.from_source(
    "polytope",
    "mchgj",
    request.to_polytope(),
    stream=False
).to_xarray()

## Plot the results


In [ ]:
from earthkit.plots.interactive import Chart


da = ds["t_2m"] - 273.15
da.name = "t_2m (°C)"

chart = Chart()
chart.line(da)

If you're viewing this on GitHub, plots are static and we export them as PNGs using `chart.show("png")`.

If you're running the notebook locally, use `chart.show()` to enable interactivity. You can hover over the chart to see precise timestamps and temperature values (°C).

In [ ]:
chart.show("png")

## Retrieve ensemble data

We can also query the entire ensemble at the same geolocation.

For ICON-CH2-EPS:

In [ ]:
import dataclasses as dc

req = dc.replace(
    request,
    number=[str(num) for num in range(1, 21)],
    model=mars.Model.ICON_CH2_EPS,
    feature={
        "type": "timeseries",
        "points": [rotated_point],
        "time_axis": "step",
        "range": {"start": 0, "end": 120},
        "axes": ["longitude", "latitude"],
    },
)

ds_ch2 = ekd.from_source("polytope", "mchgj", req.to_polytope(), stream=False).to_xarray()

For ICON-CH1-EPS:

In [ ]:
import dataclasses as dc

req = dc.replace(
    request,
    number=[str(num) for num in range(1, 11)],
    model=mars.Model.ICON_CH1_EPS,
    feature={
        "type": "timeseries",
        "points": [rotated_point],
        "time_axis": "step",
        "range": {"start": 0, "end": 33},
        "axes": ["longitude", "latitude"],
    },
)

ds_ch1 = ekd.from_source("polytope", "mchgj", req.to_polytope(), stream=False).to_xarray()

We can also request the same data from **IFS** using **ECMWF Polytope**. To access the data follow the instructions on [https://polytope.readthedocs.io/en/latest/Service/Installation/](https://polytope.readthedocs.io/en/latest/Service/Installation/) in section Authentication. Once you’ve obtained your credentials, add them to your existing `config.yaml` file, which already contains the MeteoSwiss credentials.

In [ ]:
import os

# Use your ECMWF Polytope credentials
os.environ["POLYTOPE_USER_KEY"] = config["ecmwf"]["key"]
os.environ["POLYTOPE_USER_EMAIL"] = config["ecmwf"]["user_email"]
os.environ["POLYTOPE_ADDRESS"] = "polytope.ecmwf.int"

> **Note:** For ECMWF Polytope, coordinates are **geographic** (WGS84). In the next request we pass `geo_lon` and `geo_lat`.

In [ ]:
request = {
    "class": "od",
    "stream": "enfo",
    "type": "pf",
    "date": date,
    "time": time,
    "levtype": "sfc",
    "expver": "0001",
    "domain": "g",
    "param": "167",
    "number": "1/to/50",
    "step": "0/to/160",
    "feature": {
        "type": "timeseries",
        "points": [[float(round(geo_lon, 2)), float(round(geo_lat, 2))]],
        "time_axis": "step",
        "axes": ["longitude", "latitude"]
    }
}

ds_ifs = ekd.from_source(
    "polytope",
    "ecmwf-mars",
    request,
    stream=False
)

## Plotting ensemble members of time series

In [ ]:
from earthkit.plots.interactive import Chart

TIME_FREQUENCY = "1h"
QUANTILES = [0, 0.1, 0.25, 0.5, 0.75, 0.9, 1]

chart = Chart()

# CH1
chart.box(ds_ch1, quantiles=QUANTILES, line_color="#ea5545")
chart.line(ds_ch1, aggregation="mean", line_color="grey",
           time_frequency=TIME_FREQUENCY,
           name="ICON-CH1-EPS (mean)")

# CH2
chart.box(ds_ch2, quantiles=QUANTILES, line_color="#ef9b20")
chart.line(ds_ch2, aggregation="mean", line_color="grey",
           time_frequency=TIME_FREQUENCY,
           name="ICON-CH2-EPS (mean)")

# IFS
chart.box(ds_ifs, quantiles=QUANTILES, line_color="#27aeef")
chart.line(ds_ifs, aggregation="mean", line_color="grey",
           time_frequency=TIME_FREQUENCY,
           name="IFS (mean)")

If you're viewing this on GitHub, plots are static and we export them as PNGs using `chart.show("png")`.

If you're running the notebook locally, use `chart.show()` to enable interactivity.

In [ ]:
chart.show("png")